In [2]:
from pyspark.sql import SparkSession

jars = [
    '/usr/share/aws/redshift/jdbc/RedshiftJDBC.jar',
    '/usr/share/aws/redshift/spark-redshift/lib/spark-redshift.jar',
    '/usr/share/aws/redshift/spark-redshift/lib/spark-avro.jar',
    '/usr/share/aws/redshift/spark-redshift/lib/minimal-json.jar'
]

spark = SparkSession. \
    builder. \
    appName('Redshift Integration'). \
    master('yarn'). \
    config('spark.jars', ','.join(jars)). \
    getOrCreate()

In [3]:
ghactivity = spark.read.json('s3://aigithub/landing/ghactivity/2022-06-19*')
ghactivity.createOrReplaceTempView('ghactivity')
new_repos = spark.sql("""
    SELECT
        repo.id AS repo_id,
        repo.name AS repo_name,
        actor.id AS actor_id,
        actor.login AS actor_login,
        actor.display_login AS actor_display_login,
        payload.ref_type AS ref_type,
        type,
        created_at,
        year(created_at) AS created_year,
        month(created_at) AS created_month,
        dayofmonth(created_at) AS created_dayofmonth
    FROM ghactivity
    WHERE type = 'CreateEvent'
        AND payload.ref_type = 'repository'
""")

22/07/14 00:27:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
new_repos.show()

+---------+--------------------+---------+------------------+-------------------+----------+-----------+--------------------+------------+-------------+------------------+
|  repo_id|           repo_name| actor_id|       actor_login|actor_display_login|  ref_type|       type|          created_at|created_year|created_month|created_dayofmonth|
+---------+--------------------+---------+------------------+-------------------+----------+-----------+--------------------+------------+-------------+------------------+
|505110810|   Anand4311/Project|107799449|         Anand4311|          Anand4311|repository|CreateEvent|2022-06-19T13:00:01Z|        2022|            6|                19|
|505110809|direwolf-github/e...| 10810283|   direwolf-github|    direwolf-github|repository|CreateEvent|2022-06-19T13:00:01Z|        2022|            6|                19|
|505110811|shakil033/e-comme...| 61247278|         shakil033|          shakil033|repository|CreateEvent|2022-06-19T13:00:01Z|        2022|  

In [5]:
new_repos.count()

122496

```python
# Original logic to write to files
new_repos. \
  write. \
  partitionBy('created_year', 'created_month', 'created_dayofmonth'). \
  mode('overwrite'). \
  parquet('s3://aigithub/github_dm/new_repos')
```

* Make sure the required permissions are granted on s3 to Redshift Cluster via IAM role.
* Come up with the logic to write dataframe to Redshift Table.

In [9]:
import boto3
import json
sm_client = boto3.client('secretsmanager')
secret_value = sm_client.get_secret_value(SecretId='demo/aigithub/redshift')
credentials = json.loads(secret_value['SecretString'])

username = credentials['username']
password = credentials['password']
host = credentials['host']
port = credentials['port']
database = 'github_dm'
url = f"jdbc:redshift://{host}:{port}/{database}?user={username}&password={password}"

In [8]:
new_repos. \
    write. \
    mode('append'). \
    format('io.github.spark_redshift_community.spark.redshift'). \
    option(
        'aws_iam_role', 
        'arn:aws:iam::269066542444:role/service-role/AmazonRedshift-CommandsAccessRole-20220625T110940'
    ). \
    option('url', url). \
    option('dbtable', 'public.ghrepos'). \
    option('tempdir', 's3://aigithub/temp/ghrepos'). \
    save()

22/07/14 00:46:41 WARN CredentialsLegacyConfigLocationProvider: Found the legacy config profiles file at [/home/hadoop/.aws/config]. Please move it to the latest default location [~/.aws/credentials].
22/07/14 00:46:41 WARN Utils$: The S3 bucket aigithub does not have an object lifecycle configuration to ensure cleanup of temporary files. Consider configuring `tempdir` to point to a bucket with an object lifecycle policy that automatically deletes files after an expiration period. For more information, see https://docs.aws.amazon.com/AmazonS3/latest/dev/object-lifecycle-mgmt.html


Py4JJavaError: An error occurred while calling o100.save.
: java.sql.SQLException: [Amazon](500150) Error setting/closing connection: UnknownHostException.
	at com.amazon.redshift.client.PGClient.connect(Unknown Source)
	at com.amazon.redshift.client.PGClient.<init>(Unknown Source)
	at com.amazon.redshift.core.PGJDBCConnection.connect(Unknown Source)
	at com.amazon.jdbc.common.BaseConnectionFactory.doConnect(Unknown Source)
	at com.amazon.jdbc.common.AbstractDriver.connect(Unknown Source)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverWrapper.connect(DriverWrapper.scala:46)
	at io.github.spark_redshift_community.spark.redshift.JDBCWrapper.getConnector(RedshiftJDBCWrapper.scala:232)
	at io.github.spark_redshift_community.spark.redshift.RedshiftWriter.saveToRedshift(RedshiftWriter.scala:405)
	at io.github.spark_redshift_community.spark.redshift.DefaultSource.createRelation(DefaultSource.scala:110)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(Sav